<a href="https://colab.research.google.com/github/tselane2110/ML-Specialization/blob/main/Logistic_Regression_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Logistic Regression without Regularization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
class LogisticRegression:
  def fit_data(self, x, y):
    """
    self.x -> training data (features)
    self.y -> training data (labels/target-values/actual-values)
    self.m -> no of training examples
    self.n -> no of features
    self.w -> initializing weights
    self.b = 0 # initializing bias
    self.y_hat = [] # initializing prediction's list
    """
    self.x = x
    self.y = y
    self.m = len(self.x) # no of training examples
    self.n = self.x.shape[1] # no of features
    self.w = np.zeros(self.n) # initializing weights
    self.b = 0 # initializing bias
    self.y_hat = [] # initializing prediction's list

